In [ ]:
import pandas as pd
from MicroStrategyConnection import *
from mstrio.project_objects import SuperCube

CSV_Folder_Path = 'C:/Micha/MSTR/MicroStrategyPython/DataExport/Star Schema Data Transformation/'
master_table_name = 'data'
new_master_table_ids = []
iCube_name = '_iCube_STAR_Schema_Transformation_from_CSV_File'

mstr_conn = GetMicroStrategyConnection()
iCube = SuperCube(connection=mstr_conn, name=iCube_name)
#df_Master_Table = pd.read_csv(CSV_Folder_Path + 'denormalized_retail_data.csv', sep=';', nrows=1000)
df_Master_Table = pd.read_csv(CSV_Folder_Path + 'denormalized_retail_data.csv', sep=';')

Main Function that extracts data from master table und creates lookup table for one attribute including more than one columns + create CSV File for lookup table

In [ ]:
def CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id,
                                          lookup_columns_numeric_attributes, lookup_table_name):
    
    global df_Master_Table
    global iCube
    
    df_LU_Table = df_Master_Table[lookup_columns_old].drop_duplicates()
    df_LU_Table.reset_index(drop=True, inplace=True)
    df_LU_Table.insert(0, lookup_column_id, range(1, 1 + len(df_LU_Table)))

    #We need to drop any secondary lookup columns in master table before join with new lookup table
    #First value in lookup_columns_old is the join column. Don't drop this column
    n=1
    while n < len(lookup_columns_old):
        df_Master_Table.drop(columns=[lookup_columns_old[n]], inplace=True)
        n = n + 1       

    df_Master_Table = df_Master_Table.join(df_LU_Table.set_index(lookup_columns_old[0]), on=lookup_columns_old[0])
    df_Master_Table.drop(columns=lookup_columns_old, inplace=True)

    df_Master_Table = df_Master_Table.rename(columns={lookup_columns_old[0]: lookup_columns_new[0]})

    n=0
    while n < len(lookup_columns_old):
        df_LU_Table = df_LU_Table.rename(columns={lookup_columns_old[n]: lookup_columns_new[n]})
        n = n + 1

    iCube.add_table(name=lookup_table_name, data_frame=df_LU_Table, update_policy='replace', to_attribute=lookup_columns_numeric_attributes)
    df_LU_Table.to_csv(CSV_Folder_Path + lookup_table_name + '.csv', sep=';', index=False)

Now configure each Columns Set Transformation and run the Main Function

In [ ]:
lookup_columns_old = ['Call Center']
lookup_columns_new = ['callcenter']
lookup_column_id = 'callcenter_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['callcenter_id']
lookup_table_name = 'lu_callcenter'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Category']
lookup_columns_new = ['category']
lookup_column_id = 'category_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['category_id']
lookup_table_name = 'lu_category'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Customer', 'Customer Age'] #The first column is the join column to create foreign key in master table
lookup_columns_new = ['customer', 'customer_age']
lookup_column_id = 'customer_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['customer_id', 'customer_age']
lookup_table_name = 'lu_customer'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Month', 'Month Number', 'Month Short Name'] #The first column is the join column to create foreign key in master table
lookup_columns_new = ['month', 'month_number', 'month_short_name']
lookup_column_id = 'month_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['month_id', 'month_number']
lookup_table_name = 'lu_month'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Item']
lookup_columns_new = ['item']
lookup_column_id = 'item_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['item_id']
lookup_table_name = 'lu_item'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Region']
lookup_columns_new = ['region']
lookup_column_id = 'region_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['region_id']
lookup_table_name = 'lu_region'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Subcategory']
lookup_columns_new = ['subcategory']
lookup_column_id = 'subcategory_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['subcategory_id']
lookup_table_name = 'lu_subcategory'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

In [ ]:
lookup_columns_old = ['Supplier']
lookup_columns_new = ['supplier']
lookup_column_id = 'supplier_id'
new_master_table_ids.append(lookup_column_id)
lookup_columns_numeric_attributes = ['supplier_id']
lookup_table_name = 'lu_supplier'

CreateLookupTableUpdateMasterTableWriteLookupTable(lookup_columns_old, lookup_columns_new, lookup_column_id, lookup_columns_numeric_attributes, lookup_table_name)

Tell the iCube about unchanged numberic columns in master table that need to be configured as attributes not metrics

In [ ]:
#Add numberic unchanged columns that need to be Attributes in iCube not Metrics
new_master_table_ids.append('Year')

Add Master Table to iCube und run the Cube Generation + Write Master Table CSV File too

In [ ]:
iCube.add_table(name=master_table_name, data_frame=df_Master_Table, update_policy='replace', to_attribute=new_master_table_ids)
df_Master_Table.to_csv(CSV_Folder_Path + master_table_name + '.csv', sep=';', index=False)
iCube.create()
mstr_conn.close()